# py0xcluster

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses


### Imports

In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Gathering data about most-active pools

In [87]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 100000,
    min_days_active = 2, # Not implemented. consider removing
    start_date = (2022,11,1), 
    end_date = (2023,1,10),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.get_pools_data(verbose=True)
uni3_pools

Queriying from 2022-11-01 00:00:00 to 2022-11-21 00:00:00
Queriying from 2022-11-21 00:00:00 to 2022-12-11 00:00:00
Queriying from 2022-12-11 00:00:00 to 2022-12-31 00:00:00
Queriying from 2022-12-31 00:00:00 to 2023-01-10 00:00:00
7526 lquidity pools snapshots retrieved


,timestamp,dailyVolumeUSD,pool.symbol,pool.totalValueLockedUSD,pool.protocol.name,pool.protocol.network,pool.name,pool.id,token0.symbol,token0.lastPriceUSD,token1.symbol,token1.lastPriceUSD
0,2022-11-09 23:59:59,2.257639e+09,USD Coin/Wrapped Ether,1.787315e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1337.925269
1,2022-11-08 23:59:59,1.954942e+09,USD Coin/Wrapped Ether,1.787315e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1337.925269
2,2022-11-10 23:59:59,1.728625e+09,USD Coin/Wrapped Ether,1.787315e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1337.925269
3,2022-11-11 23:59:59,9.670028e+08,USD Coin/Wrapped Ether,1.787315e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1337.925269
4,2022-11-04 23:59:59,8.497930e+08,USD Coin/Wrapped Ether,1.787315e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1337.925269
...,...,...,...,...,...,...,...,...,...,...,...,...
7521,2022-12-30 23:04:11,1.021630e+05,FunFair/Wrapped Ether,3.399031e+06,Uniswap V3,MAINNET,Uniswap V3 FunFair/Wrapped Ether 0.01%,0x29f0096512b4af1d689c1a11a867a6e707a8dcde,FUN,0.006302,WETH,1337.925269
7522,2023-01-04 23:45:23,1.020119e+05,Dogechain Token/Wrapped Ether,1.826047e+06,Uniswap V3,MAINNET,Uniswap V3 Dogechain Token/Wrapped Ether 1%,0x498498fa386ef2860e7abf8c60254580c8c41ec5,DC,0.001106,WETH,1337.925269
7523,2023-01-08 23:42:23,1.019716e+05,Index/Wrapped Ether,2.708347e+04,Uniswap V3,MAINNET,Uniswap V3 Index/Wrapped Ether 1%,0x8c13148228765ba9e84eaf940b0416a5e349a5e7,INDEX,0.000000,WETH,1337.925269
7524,2023-01-04 23:59:47,1.017113e+05,PulseBitcoin/USD Coin,8.684875e+04,Uniswap V3,MAINNET,Uniswap V3 PulseBitcoin/USD Coin 1%,0xa5ef2a6bbe8852bd6fd2ef6ab9bb45081a6f531c,PLSB,0.000000,USDC,1.000000


### Filtering pools
    - more exactly filtering snapshots before processing

In [88]:
min_TVL = 500000

uni3_pools = uni3pools_selector.remove_illiquid_pools(uni3_pools, min_TVL)
uni3_pools = uni3pools_selector.remove_stable_pools(uni3_pools)

# After this point uni3_pools dataframe changes structure to become an aggregation
# (only run once or query again raw data)
uni3_pools = uni3pools_selector.aggregate_snapshots(uni3_pools)

uni3_pools

2839 illiquid pools snapshots (over 7526) have been removed 
730 stable pools snapshots (over 4687) have been removed


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.626963e+08,1.787315e+08,1.000000,1337.925269,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,3.683370e+07,1.114904e+08,17418.882366,1337.925269,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.380961e+07,3.468095e+07,1337.925269,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.3%,1.788214e+07,1.355260e+08,1.000000,1337.925269,Uniswap V3,MAINNET,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH
4,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.082587e+07,5.052765e+06,1.000000,1337.925269,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
...,...,...,...,...,...,...,...,...,...,...
104,Uniswap V3 Tether USD/AVINOC Token 0.3%,1.180537e+05,5.663719e+06,1.000000,0.228010,Uniswap V3,MAINNET,0x2eb8f5708f238b0a2588f044ade8dea7221639ab,USDT,AVINOC
105,Uniswap V3 Status Network Token/Wrapped Ether ...,1.155646e+05,1.470164e+06,0.021328,1337.925269,Uniswap V3,MAINNET,0x084b5191bd08412952337b1108b6e5942418928f,SNT,WETH
106,Uniswap V3 USD Coin/AVINOC Token 0.3%,1.131157e+05,6.873842e+06,1.000000,0.228010,Uniswap V3,MAINNET,0x04a2004b2032fef2ba93f40b0e34d26ab7b00120,USDC,AVINOC
107,Uniswap V3 Green/Wrapped Ether 0.05%,1.081729e+05,6.052408e+05,0.002039,1337.925269,Uniswap V3,MAINNET,0x4efc9e2e3e77732ce2f9612b8f050082c01688bd,GREEN,WETH


In [81]:
pools_description = uni3_pools.drop_duplicates(subset='pool.name', keep='first')
pools_description = pools_description.select_dtypes(include='string').set_index('pool.symbol')

pools_stats = uni3_pools.groupby('pool.name').agg('median', numeric_only=True).sort_values('dailyVolumeUSD', ascending=False)
pools_stats = pools_stats.merge(pools_description, on='pool.name') 
pools_stats.head(50)

,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.626963e+08,1.787286e+08,1.000000e+00,1337.925269,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,3.683370e+07,1.114817e+08,1.741731e+04,1337.925269,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.380961e+07,3.468091e+07,1.337925e+03,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.3%,1.788214e+07,1.355260e+08,1.000000e+00,1337.925269,Uniswap V3,MAINNET,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH
4,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.082587e+07,4.985062e+06,1.000000e+00,1337.925269,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
5,Uniswap V3 Wrapped BTC/Wrapped Ether 0.3%,1.060664e+07,1.345862e+08,1.741731e+04,1337.925269,Uniswap V3,MAINNET,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC,WETH
6,Uniswap V3 Dai Stablecoin/Wrapped Ether 0.05%,7.377514e+06,6.317675e+06,1.000000e+00,1337.925269,Uniswap V3,MAINNET,0x60594a405d53811d3bc4766596efd80fd545a270,DAI,WETH
7,Uniswap V3 Wrapped BTC/USD Coin 0.05%,6.400518e+06,3.734143e+06,1.741731e+04,1.000000,Uniswap V3,MAINNET,0x9a772018fbd77fcd2d25657e5c547baff3fd7d16,WBTC,USDC
8,Uniswap V3 ChainLink Token/Wrapped Ether 0.3%,5.949516e+06,1.962222e+07,6.172339e+00,1337.925269,Uniswap V3,MAINNET,0xa6cc3c2531fdaa6ae1a3ca84c2855806728693e8,LINK,WETH
9,Uniswap V3 Wrapped Ether/Tether USD 0.3%,5.451259e+06,5.162657e+07,1.337925e+03,1.000000,Uniswap V3,MAINNET,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH,USDT


In [332]:
uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).head(50)

/tmp/ipykernel_58247/2384805637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).head(50)


,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD
pool.symbol,,,,
USD Coin/Wrapped Ether,4.247551e+07,1.350663e+08,1.000000,1296.537802
USD Coin/Tether USD,3.287978e+07,1.194780e+08,1.000000,1.000000
Wrapped BTC/Wrapped Ether,2.804087e+07,1.089433e+08,17224.761416,1296.537802
Wrapped Ether/Tether USD,2.175894e+07,3.462967e+07,1296.537802,1.000000
Dai Stablecoin/USD Coin,6.518340e+06,2.352720e+08,1.000000,1.000000
Dai Stablecoin/Wrapped Ether,6.353363e+06,6.181654e+06,1.000000,1296.537802
Wrapped BTC/USD Coin,5.853777e+06,2.224176e+07,17224.761416,1.000000
ChainLink Token/Wrapped Ether,5.086765e+06,1.927886e+07,6.092683,1296.537802
Rocket Pool/Wrapped Ether,4.813042e+06,2.114466e+05,0.000000,1296.537802
